In [ ]:
import pandas as pd
from transformers import pipeline
import torch

In [ ]:
model=pipeline("sentiment-analysis",
                      "blanchefort/rubert-base-cased-sentiment", device=0 if torch.cuda.is_available() else -1,
                 framework="pt", max_length=max_length)

In [ ]:
df_test = pd.read_csv('df_feedback_filt.csv', encoding="utf-8")

In [ ]:
df_test

,Отзыв
0,-
1,"Спасибо за условия! Чисто, удобно! Хорошая еда..."
2,Все очень понравилось. Очень комфортная атмос...
3,"Очень чисто, тихо, уютно. Единственное, смущаю..."
4,"Как всегда чисто, приятный персонал, вкусная е..."
...,...
3377,"Очень страшно рекомендовать друзьям, потому чт..."
3378,Спасибо большое за внимательное доброжелательн...
3379,Приятен уровень Сбербанка! очень приветливый п...
3380,"Люблю это место. Жаль, что закрыты спортивный ..."


In [ ]:
df_test.rename(columns={'Отзыв': 'text'},inplace=True)

In [ ]:
df_test

,text
0,-
1,"Спасибо за условия! Чисто, удобно! Хорошая еда..."
2,Все очень понравилось. Очень комфортная атмос...
3,"Очень чисто, тихо, уютно. Единственное, смущаю..."
4,"Как всегда чисто, приятный персонал, вкусная е..."
...,...
3377,"Очень страшно рекомендовать друзьям, потому чт..."
3378,Спасибо большое за внимательное доброжелательн...
3379,Приятен уровень Сбербанка! очень приветливый п...
3380,"Люблю это место. Жаль, что закрыты спортивный ..."


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
import pandas as pd
max_length = 1024
model_checkpoint = "blanchefort/rubert-base-cased-sentiment"
# Загрузка модели для анализа тональности текста
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
# Установка максимальной длины последовательности для токенизатора и модели
tokenizer.model_max_length = max_length
model.config.max_position_embeddings = max_length
model.base_model.embeddings.position_ids = torch.arange(max_length).expand((1, -1))
orig_pos_emb = model.base_model.embeddings.position_embeddings.weight
model.base_model.embeddings.position_embeddings.weight = torch.nn.Parameter(torch.cat((orig_pos_emb, orig_pos_emb)))
# Создание пайплайна для анализа тональности текста
model = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
lst = []
for i in df_test["text"]:
  lst.append(model(str(i))[0]["label"])
df_test["Sentinent"]=pd.DataFrame(lst)
df_test

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

,text,Sentinent
0,-,NEUTRAL
1,"Спасибо за условия! Чисто, удобно! Хорошая еда...",POSITIVE
2,Все очень понравилось. Очень комфортная атмос...,POSITIVE
3,"Очень чисто, тихо, уютно. Единственное, смущаю...",POSITIVE
4,"Как всегда чисто, приятный персонал, вкусная е...",POSITIVE
...,...,...
3377,"Очень страшно рекомендовать друзьям, потому чт...",POSITIVE
3378,Спасибо большое за внимательное доброжелательн...,POSITIVE
3379,Приятен уровень Сбербанка! очень приветливый п...,POSITIVE
3380,"Люблю это место. Жаль, что закрыты спортивный ...",POSITIVE


In [ ]:
df_test.head(59)

,text,Sentinent
0,-,NEUTRAL
1,"Спасибо за условия! Чисто, удобно! Хорошая еда...",POSITIVE
2,Все очень понравилось. Очень комфортная атмос...,POSITIVE
3,"Очень чисто, тихо, уютно. Единственное, смущаю...",POSITIVE
4,"Как всегда чисто, приятный персонал, вкусная е...",POSITIVE
5,Прекрасное место для отдыха и развития. Место ...,POSITIVE
6,стабильно 1 раз в год бываю в кампусе. каждый ...,POSITIVE
7,"Все супер, территория прекрасна , персонал при...",POSITIVE
8,"Все очень прекрасно, весело, здорово, позитивн...",POSITIVE
9,Спасибо за сервис! Просьба завершить как можно...,NEUTRAL


In [ ]:
df_test['Sentinent'].to_csv('/content/df_pred.csv', index = False)

In [ ]:
df_test.head(10)

,text,Sentinent
0,-,NEUTRAL
1,"Спасибо за условия! Чисто, удобно! Хорошая еда...",POSITIVE
2,Все очень понравилось. Очень комфортная атмос...,POSITIVE
3,"Очень чисто, тихо, уютно. Единственное, смущаю...",POSITIVE
4,"Как всегда чисто, приятный персонал, вкусная е...",POSITIVE
5,Прекрасное место для отдыха и развития. Место ...,POSITIVE
6,стабильно 1 раз в год бываю в кампусе. каждый ...,POSITIVE
7,"Все супер, территория прекрасна , персонал при...",POSITIVE
8,"Все очень прекрасно, весело, здорово, позитивн...",POSITIVE
9,Спасибо за сервис! Просьба завершить как можно...,NEUTRAL


In [ ]:
predicted_labels = df_test['Sentinent']
predicted_labels

0        NEUTRAL
1       POSITIVE
2       POSITIVE
3       POSITIVE
4       POSITIVE
          ...   
3377    POSITIVE
3378    POSITIVE
3379    POSITIVE
3380    POSITIVE
3381    NEGATIVE
Name: Sentinent, Length: 3382, dtype: object

In [ ]:
true_labels = pd.read_csv('df_re_new_new_d.csv', encoding="utf-8")
# true_labels['estimate;;'] = true_labels['estimate;;'].str.replace(';;', '')
true_labels = true_labels['estimate']
true_labels

0        NEUTRAL
1       POSITIVE
2       POSITIVE
3       POSITIVE
4       POSITIVE
          ...   
3377    POSITIVE
3378    POSITIVE
3379    POSITIVE
3380    POSITIVE
3381    POSITIVE
Name: estimate, Length: 3382, dtype: object

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Предсказанные значения модели
# predicted_labels = [...]  # Предсказанные метки
# true_labels = [...]       # Фактические метки

# Вычисление метрик
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='weighted')


# Вывод результатов
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.8160851567120048
Precision: 0.8334498747945419
Recall: 0.6658209753612802
F1-score: 0.8235563750313125


NLI

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

def predict_zero_shot_df(dataset, label_texts, model, tokenizer, label='entailment', normalize=True):
    predictions = []
    for index, row in dataset.iterrows():
        text = row['text']
        tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
        with torch.inference_mode():
            result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
        proba = result[:, model.config.label2id[label]].cpu().numpy()
        if normalize:
            proba /= sum(proba)

        # Добавляем логику для определения тональности
        max_proba_index = proba.argmax()
        if max_proba_index == 0:
            sentiment = 'POSITIVE'
        elif max_proba_index == 1:
            sentiment = 'NEGATIVE'
        elif max_proba_index == 2:
            sentiment = 'NEUTRAL'
        else:
            sentiment = 'не определен'

        predictions.append({
            'Текст': text,
            'Результат': sentiment,
            'p1': proba[0],
            'p2': proba[1],
            'p3': proba[2]
        })

    return pd.DataFrame(predictions)

# Пример использования функции
dataset = df_test  # Предполагается, что у вас есть DataFrame с данными, где колонка 'text' содержит тексты отзывов
classes = ['позитивный', 'негативный', 'нейтральный']

predictions_df = predict_zero_shot_df(dataset, classes, model, tokenizer)
# print(predictions_df)

In [ ]:
predictions_df

,Текст,Результат,p1,p2,p3
0,-,POSITIVE,0.477460,0.310849,0.211691
1,"Спасибо за условия! Чисто, удобно! Хорошая еда...",POSITIVE,0.939052,0.002156,0.058792
2,Все очень понравилось. Очень комфортная атмос...,POSITIVE,0.841551,0.001595,0.156854
3,"Очень чисто, тихо, уютно. Единственное, смущаю...",NEUTRAL,0.236566,0.031971,0.731463
4,"Как всегда чисто, приятный персонал, вкусная е...",POSITIVE,0.881521,0.001213,0.117266
...,...,...,...,...,...
3377,"Очень страшно рекомендовать друзьям, потому чт...",NEGATIVE,0.048313,0.803443,0.148244
3378,Спасибо большое за внимательное доброжелательн...,POSITIVE,0.918223,0.006888,0.074889
3379,Приятен уровень Сбербанка! очень приветливый п...,POSITIVE,0.914201,0.000953,0.084846
3380,"Люблю это место. Жаль, что закрыты спортивный ...",POSITIVE,0.878956,0.004423,0.116621


## Суммаризация

In [ ]:
import pandas as pd
import string

# Пример DataFrame
df = predictions_df

# Удаление знаков пунктуации
# punctuation_marks = [':','..','...','!','?','-', '..', '...', '@', ';', '/', "''", "``",'№','«','»','–','$','&','*','(',')','+']
# def preprocess_1(text, punctuation_marks):
#     text_without_punctuation = text.translate(str.maketrans('', '', ''.join(punctuation_marks)))
#     return text_without_punctuation

# # Применим функцию к столбцу 'текст' в датасете
# df['text_new'] = df['Текст'].apply(lambda x: preprocess_1(x, punctuation_marks))

# # Добавление точки в конце каждой строки
df['text_new'] = df['Текст']
# .str.lower() + '.'

In [ ]:
df['text_new'][1]

'Спасибо за условия! Чисто, удобно! Хорошая еда, хорошие возможности для учебы и отдыха! Территория - безумно ухоженная и красивая! Спасибо!'

In [ ]:
!pip install sumy
!pip install pymorphy3
import nltk
nltk.download('punkt')

In [ ]:

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
import pandas as pd

def summarize_dataframe(df, positive_sentences_count=1, negative_sentences_count=1, neutral_sentences_count=1):
    # Разделяем DataFrame на три группы по значению теста
    positive_df = df[df['Результат'] == 'POSITIVE']
    negative_df = df[df['Результат'] == 'NEGATIVE']
    neutral_df = df[df['Результат'] == 'NEUTRAL']

    # Суммаризируем предложения для положительных, отрицательных и нейтральных значений теста
    positive_summary = summarize(positive_df, positive_sentences_count)
    negative_summary = summarize(negative_df, negative_sentences_count)
    neutral_summary = summarize(neutral_df, neutral_sentences_count)
    return positive_summary, negative_summary, neutral_summary

def summarize(df, sentences_count):
    # Объединяем все строки в один текст
    text = ''.join(df['text_new'])

    # Инициализация парсера и токенизатора
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))

    # Инициализация суммаризатора на основе TextRank
    summarizer = TextRankSummarizer()

    # Получение суммаризации текста
    summary = summarizer(parser.document, sentences_count=sentences_count)

    # Возвращаем суммаризацию в виде списка строк
    return '.'.join([str(sentence) for sentence in summary])

# Пример DataFrame df с колонками 'Предложение', 'Класс' и 'тест'


# Создаем DataFrame для сохранения результатов суммаризации
result_summary = pd.DataFrame()

# Суммаризируем предложения для каждого значения теста
positive_summary, negative_summary, neutral_summary = summarize_dataframe(df, positive_sentences_count=1, negative_sentences_count=1, neutral_sentences_count=1)

# Создаем DataFrame с результатом суммаризации
result_summary = pd.DataFrame({
    'positive': [positive_summary],
    'negative': [negative_summary],
    'neutral': [neutral_summary]
})

result_summary

,positive,negative,neutral
0,Столовая кофе поинты полностью соответствовали...,"Все-таки в выходные бывают ""Семейные выходные""...",А так же в рамках умных выходных реальное коли...


In [ ]:
result_summary['neutral'][0]

'А так же в рамках умных выходных реальное количество времени гораздо меньше заявленных.Забыла в номере личную вещь, обратилась в гостиницу на следующий день и мне сказали, что не нашли.До 7 лет детям нечего делать (бассейн и спа нет почему то) Питание на 3  Номер люкс не годится для семей с разнополыми детьми (2 взр и 2 детей)  Двуместные кровати неширокие и с матрасов можно скатитьсяВ двуспальный номер можно поставить 2 креслаПредложение: в столовой не убирать еду за 5 мин до закрытия, не правильно.'

In [ ]:
result_summary.to_csv('/content/result_summary.csv', index = False)

In [ ]:
predictions_df['Результат'].to_csv('/content/predictions_df.csv', index = False)

In [ ]:
predictions_df = pd.read_csv('predictions_df.csv', encoding="utf-8")

In [ ]:
value_counts = predictions_df['Результат'].value_counts()

# Создание нового DataFrame
new_df = pd.DataFrame({
    'negative': [value_counts.get('NEGATIVE', 0)],
    'positive': [value_counts.get('POSITIVE', 0)],
    'neutral': [value_counts.get('NEUTRAL', 0)]
})

In [ ]:
new_df.to_csv('/content/predictions_cnt.csv', index = False)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
predictions_df.head(3000)

In [ ]:
count_values = predictions_df.groupby('Результат')['Результат'].count()

In [ ]:
count_values

Результат
NEGATIVE     314
NEUTRAL      422
POSITIVE    2646
Name: Результат, dtype: int64

Оценка NLI

In [ ]:
predicted_labels = predictions_df['Результат']
predicted_labels

0       POSITIVE
1       POSITIVE
2       POSITIVE
3        NEUTRAL
4       POSITIVE
          ...   
3377    NEGATIVE
3378    POSITIVE
3379    POSITIVE
3380    POSITIVE
3381    POSITIVE
Name: Результат, Length: 3382, dtype: object

In [ ]:
true_labels = pd.read_csv('df_re_new_new_d.csv', encoding="utf-8")
# true_labels['estimate;;'] = true_labels['estimate;;'].str.replace(';;', '')
true_labels = true_labels['estimate']
true_labels

0        NEUTRAL
1       POSITIVE
2       POSITIVE
3       POSITIVE
4       POSITIVE
          ...   
3377    POSITIVE
3378    POSITIVE
3379    POSITIVE
3380    POSITIVE
3381    POSITIVE
Name: estimate, Length: 3382, dtype: object

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Предсказанные значения модели
# predicted_labels = [...]  # Предсказанные метки
# true_labels = [...]       # Фактические метки

# Вычисление метрик
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='weighted')


# Вывод результатов
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8486102897693673
Precision: 0.8488760688251199
Recall: 0.6868003206674221
F1-score: 0.8486151742987612
